In [102]:
import pandas as pd
import numpy as np
import random

#Author: Jian Gao

# <font color = "red"> Part 1</font>

## Load probability from given tables

In [103]:
transition_url = "lab3data/transitionProbs.csv"
data_url = "lab3data/testData.csv"
observation_url = "lab3data/observationProbs.csv"

transition_df = pd.read_csv(transition_url)
observation_df = pd.read_csv(observation_url)
hidden_variables = np.array(transition_df)[:2,0]
observations = np.array(observation_df)[:,0]
data = np.array(pd.read_csv(data_url))[:,1:6]
start_prob = {}
stop_prob = {}
transition_prob = {}
emission_prob = {}

for i in range(len(hidden_variables)):
    y = hidden_variables[i]
    start_prob[y] = np.array(transition_df[['START']])[i][0] 
    stop_prob[y] = np.array(transition_df)[2][i+1]
    temp_dict1 = {}
    temp_dict2= {}
    transition_prob[y] = temp_dict1
    emission_prob[y] = temp_dict2
    for a in range(len(hidden_variables)):
        temp_dict1[hidden_variables[a]] = np.array(transition_df)[i][a+1]
    for b in range(len(observations)):
        temp_dict2[observations[b]] = np.array(observation_df)[:,i+1][b]
   




## Define viterbi algorithm and generate arg max path and table 

In [104]:
def get_key (dict, val):
    for key, value in dict.items():
        if val == value:
            return key

def viterbi_algorithm(observations, hidden_variables, start_prob, transition_prob, emission_prob, stop_prob):
    print ("For obeservations: " , observations)
    table = [{}]
    path = {}
    for y in hidden_variables:
        table[0][y] = start_prob[y] * emission_prob[y][observations[0]]
        path[y] = [y]
    
    for n in range(1,len(observations)):
        table.append({})
        temp_path = {}
        for y in hidden_variables:
            prob = {}
            for x in hidden_variables:
                original_p = table[n-1][x]
                transition_p = transition_prob[x][y]
                emission_p = emission_prob[y][observations[n]]
                prob[x] = original_p * transition_p * emission_p
            
            probability = max(prob.values())
            variable = get_key(prob, probability)
            table[n][y] = probability;
            temp_path[y] = path[variable] + [y]
        
        path = temp_path
    
    last_layer_prob = {}
    for x in hidden_variables:
        last_layer_prob[x] = table[len(observations)-1][x] * stop_prob[x]
    
    probability = max(last_layer_prob.values())
    variable = get_key(last_layer_prob, probability)
    return path[variable]



### Get result of <font color = "red">Viterbi </font>algorithm

In [105]:
for i in range(len(data)):
    obs = data[i][data[i] != 0]
    print("Solution by Viterbi Algorithm:",  viterbi_algorithm(obs,
                                                               hidden_variables,
                                                               start_prob,
                                                               transition_prob,
                                                               emission_prob,
                                                               stop_prob))
    print()
    

For obeservations:  [2 3 3 2 3]
Solution by Viterbi Algorithm: ['H', 'H', 'H', 'H', 'H']

For obeservations:  [2 3 2 2]
Solution by Viterbi Algorithm: ['H', 'H', 'H', 'H']

For obeservations:  [3 1 3 3 1]
Solution by Viterbi Algorithm: ['C', 'C', 'C', 'C', 'C']

For obeservations:  [2 1 1]
Solution by Viterbi Algorithm: ['C', 'C', 'C']

For obeservations:  [1 1 1 2 3]
Solution by Viterbi Algorithm: ['C', 'C', 'C', 'C', 'C']

For obeservations:  [1 3 1 1]
Solution by Viterbi Algorithm: ['C', 'C', 'C', 'C']

For obeservations:  [3 2 3]
Solution by Viterbi Algorithm: ['H', 'H', 'H']

For obeservations:  [2 2 1 1]
Solution by Viterbi Algorithm: ['H', 'H', 'C', 'C']

For obeservations:  [1 3 2 3 3]
Solution by Viterbi Algorithm: ['C', 'H', 'H', 'H', 'H']

For obeservations:  [1 1 1]
Solution by Viterbi Algorithm: ['C', 'C', 'C']



## Conclusion for Part1
### Part I uses Viterbi Algorithm, the result is resonable according to the transition table and emission table. Also, it can handle all kinds of observations input in the file.

# <font color = "red"> Part 2 </font>

In [106]:
def normalize_transition(hidden_variables, transition_prob):
    for y in hidden_variables:
        total_prob = 0
        for x in hidden_variables:
            total_prob += transition_prob[x][y]
        for var in hidden_variables:
            transition_prob[var][y] = transition_prob[var][y]/total_prob
   
    return transition_prob
            

# Regard "C" as an evidence variable
def weighted_sampling(hidden_variables, times, observation, start_prob, transition_prob, emission_prob):
    probability = random.random()
    samples = {}
    states = []
    new_transition_prob = normalize_transition(hidden_variables, transition_prob)
    for i in range(times):
        state = ""
        if probability < 0.5:
            state = "C"
        else:
            state = "H"
        states.append(state)
        weight = emission_prob[state][observation[0]]
        for x in range(1, len(observation)):
            transition_p = new_transition_prob["C"][state]
            probability = random.random()
            if probability < transition_p:
                state = "C"
            else:
                state = "H"
            states.append(state)   
            weight *= emission_prob[state][observation[x]]
        if tuple(states) in samples:
            samples[tuple(states)] += weight
        else:
            samples[tuple(states)] = weight
        states = []
    
    max_weight = max(samples.values())
    max_states = get_key(samples, max_weight)
    return max_states
            
            
        
        
    

### Get result of <font color = "red">Likelihood Weighted Sampling </font>algorithm

In [107]:
for i in range(len(data)):
    obs = data[i][data[i] != 0]
    print ("For observations: ", obs)
    print("Solution by Weighted Sampling Algorithm after 10000 samples:",  weighted_sampling(hidden_variables,
                                                                                        10000,
                                                                                         obs,
                                                                               start_prob,
                                                                           transition_prob,
                                                                               emission_prob))
    print()
    

For observations:  [2 3 3 2 3]
Solution by Weighted Sampling Algorithm after 10000 samples: ('H', 'H', 'H', 'H', 'H')

For observations:  [2 3 2 2]
Solution by Weighted Sampling Algorithm after 10000 samples: ('H', 'H', 'H', 'H')

For observations:  [3 1 3 3 1]
Solution by Weighted Sampling Algorithm after 10000 samples: ('C', 'C', 'C', 'C', 'C')

For observations:  [2 1 1]
Solution by Weighted Sampling Algorithm after 10000 samples: ('C', 'C', 'C')

For observations:  [1 1 1 2 3]
Solution by Weighted Sampling Algorithm after 10000 samples: ('C', 'C', 'C', 'C', 'C')

For observations:  [1 3 1 1]
Solution by Weighted Sampling Algorithm after 10000 samples: ('C', 'C', 'C', 'C')

For observations:  [3 2 3]
Solution by Weighted Sampling Algorithm after 10000 samples: ('H', 'H', 'H')

For observations:  [2 2 1 1]
Solution by Weighted Sampling Algorithm after 10000 samples: ('H', 'H', 'C', 'C')

For observations:  [1 3 2 3 3]
Solution by Weighted Sampling Algorithm after 10000 samples: ('C',

## Conclusion for Part II
### After 10000 samples, the result of Likelihood Weighted Sampling matches the result of Viterbi Algorithm. Also, I normalized the transition probability according to the instruction on Carmen. So, the result are more accurate.

## <font color = "red"> Bonus</font>
### Define forward-backward algorithm

In [108]:
def forward_algorithm(observations, hidden_variables, start_prob, transition_prob, emission_prob, end_state_num, end_state):
    forward = []
    previous = {}
    previous_sum = 0
    for i in range(end_state_num):
        current = {}
        for y in hidden_variables:
            if i == 0:
                previous_sum = start_prob[y]
            else:
                for x in hidden_variables:
                    previous_sum += previous[x]*transition_prob[x][y]
            current[y] = emission_prob[y][observations[i]] * previous_sum
            
        forward.append(current)
        previous = current
    
    return forward[end_state_num-1][end_state]
#sum(forward[end_state_num-1].values())
        
def backward_algorithm(observations, hidden_variables, start_prob, transition_prob, emission_prob, end_state_num, end_state):
    backward = []
    previous = {}
    previous_sum = 0
    for i in range(end_state_num,len(observations)):
        current = {}
        for y in hidden_variables:
            if i == (end_state_num):
                previous_sum = 1
            else:
                for x in hidden_variables:
                    previous_sum += previous[x]*transition_prob[x][y]
                    
            current[y] = emission_prob[y][observations[i]] * previous_sum
            
        backward.append(current)
        previous = current
    
    return backward[len(observations) - end_state_num - 1][end_state]
#sum(backward[len(observations) - end_state - 1].values())


def calculate_prob(end_state_num, end_state, data, hidden_variables, start_prob, transition_prob, emission_prob):
    forward = forward_algorithm(data, hidden_variables, start_prob, transition_prob, emission_prob, end_state_num, end_state)
    backward = backward_algorithm(data, hidden_variables, start_prob, transition_prob, emission_prob, end_state_num, end_state)
    numerator = forward * backward
    sum = 0
    for y in hidden_variables:
        sum += forward_algorithm(data, hidden_variables, start_prob, transition_prob, emission_prob, end_state_num, y) * backward_algorithm(data, hidden_variables, start_prob, transition_prob, emission_prob, end_state_num, y)
    return numerator/sum

## Get result, I suppose to predict the first state

In [109]:
for i in range(len(data)):
    obs = data[i][data[i] != 0]
    print ("For observations: ", obs)
    predict_H = calculate_prob(1, "H", obs, hidden_variables, start_prob, transition_prob, emission_prob)
    predict_C = calculate_prob(1, "C", obs, hidden_variables, start_prob, transition_prob, emission_prob)
    print ("Probability of p(x1='H'|x) is: ", predict_H)
    print ("Probability of p(x1='C'|x) is: ", predict_C)
    if predict_H > predict_C:
        print("By comparison, the state is more likely to be: H")
    else:
        print("By comparison, the state is more likely to be: C")
    print ()
        

For observations:  [2 3 3 2 3]
Probability of p(x1='H'|x) is:  0.9193578578212126
Probability of p(x1='C'|x) is:  0.08064214217878742
By comparison, the state is more likely to be: H

For observations:  [2 3 2 2]
Probability of p(x1='H'|x) is:  0.9493163143298593
Probability of p(x1='C'|x) is:  0.05068368567014067
By comparison, the state is more likely to be: H

For observations:  [3 1 3 3 1]
Probability of p(x1='H'|x) is:  0.0009412414458720964
Probability of p(x1='C'|x) is:  0.999058758554128
By comparison, the state is more likely to be: C

For observations:  [2 1 1]
Probability of p(x1='H'|x) is:  0.0011586296178699405
Probability of p(x1='C'|x) is:  0.99884137038213
By comparison, the state is more likely to be: C

For observations:  [1 1 1 2 3]
Probability of p(x1='H'|x) is:  0.0009788415151046552
Probability of p(x1='C'|x) is:  0.9990211584848954
By comparison, the state is more likely to be: C

For observations:  [1 3 1 1]
Probability of p(x1='H'|x) is:  9.985891610249127e-08


## Add a new State "Warm" to the table

In [110]:
new_transition_url = "lab3data/Transition-3states.csv"
new_observation_url = "lab3data/Emission-3states.csv"

new_transition_df = pd.read_csv(new_transition_url)
new_observation_df = pd.read_csv(new_observation_url)
new_hidden_variables = np.array(new_transition_df)[:3,0]
new_observations = np.array(new_observation_df)[:,0]

new_start_prob = {}
new_stop_prob = {}
new_transition_prob = {}
new_emission_prob = {}
for i in range(len(new_hidden_variables)):
    y = new_hidden_variables[i]
    new_start_prob[y] = np.array(new_transition_df[['START']])[i][0] 
    new_stop_prob[y] = np.array(new_transition_df)[2][i+1]
    temp_dict1 = {}
    temp_dict2= {}
    new_transition_prob[y] = temp_dict1
    new_emission_prob[y] = temp_dict2
    for a in range(len(new_hidden_variables)):
        temp_dict1[new_hidden_variables[a]] = np.array(new_transition_df)[i][a+1]
    for b in range(len(observations)):
        temp_dict2[new_observations[b]] = np.array(new_observation_df)[:,i+1][b]
   




### Get result of <font color = "red">Viterbi </font>algorithm

In [111]:
for i in range(len(data)):
    obs = data[i][data[i] != 0]
    print("Solution by Viterbi Algorithm:",  viterbi_algorithm(obs,
                                                               new_hidden_variables,
                                                               new_start_prob,
                                                               new_transition_prob,
                                                               new_emission_prob,
                                                               new_stop_prob))
    print()
    

For obeservations:  [2 3 3 2 3]
Solution by Viterbi Algorithm: ['H', 'H', 'H', 'H', 'H']

For obeservations:  [2 3 2 2]
Solution by Viterbi Algorithm: ['H', 'H', 'H', 'H']

For obeservations:  [3 1 3 3 1]
Solution by Viterbi Algorithm: ['C', 'C', 'C', 'C', 'C']

For obeservations:  [2 1 1]
Solution by Viterbi Algorithm: ['W', 'C', 'C']

For obeservations:  [1 1 1 2 3]
Solution by Viterbi Algorithm: ['C', 'C', 'C', 'W', 'W']

For obeservations:  [1 3 1 1]
Solution by Viterbi Algorithm: ['C', 'C', 'C', 'C']

For obeservations:  [3 2 3]
Solution by Viterbi Algorithm: ['H', 'H', 'H']

For obeservations:  [2 2 1 1]
Solution by Viterbi Algorithm: ['W', 'W', 'C', 'C']

For obeservations:  [1 3 2 3 3]
Solution by Viterbi Algorithm: ['C', 'W', 'W', 'W', 'W']

For obeservations:  [1 1 1]
Solution by Viterbi Algorithm: ['C', 'C', 'C']



### Define new weighed_sampling algorithm to handle 3 states

In [112]:
def new_weighted_sampling(hidden_variables, times, observation, start_prob, transition_prob, emission_prob):
    probability = random.random()
    samples = {}
    states = []
    new_transition_prob = normalize_transition(hidden_variables, transition_prob)
    for i in range(times):
        state = ""
        if probability < 0.33:
            state = "C"
        else:
                state = random.choice(["H", "W"])
        states.append(state)
        weight = emission_prob[state][observation[0]]
        for x in range(1, len(observation)):
            transition_p = new_transition_prob["C"][state]
            probability = random.random()
            if probability < transition_p:
                state = "C"
            else:
                state = random.choice(["H", "W"])
            states.append(state)   
            weight *= emission_prob[state][observation[x]]
        if tuple(states) in samples:
            samples[tuple(states)] += weight
        else:
            samples[tuple(states)] = weight
        states = []
    
    max_weight = max(samples.values())
    max_states = get_key(samples, max_weight)
    return max_states
            

### Get result of <font color = "red">Likelihood Weighted Sampling </font>algorithm

In [113]:
for i in range(len(data)):
    obs = data[i][data[i] != 0]
    print ("For observations: ", obs)
    print("Solution by Weighted Sampling Algorithm after 50000 samples:",  new_weighted_sampling(new_hidden_variables,
                                                                                                            50000,
                                                                                                              obs,
                                                                                                   new_start_prob,
                                                                                              new_transition_prob,
                                                                                                new_emission_prob))
    print()
    

For observations:  [2 3 3 2 3]
Solution by Weighted Sampling Algorithm after 50000 samples: ('H', 'H', 'H', 'H', 'H')

For observations:  [2 3 2 2]
Solution by Weighted Sampling Algorithm after 50000 samples: ('H', 'H', 'H', 'W')

For observations:  [3 1 3 3 1]
Solution by Weighted Sampling Algorithm after 50000 samples: ('C', 'C', 'C', 'C', 'C')

For observations:  [2 1 1]
Solution by Weighted Sampling Algorithm after 50000 samples: ('W', 'C', 'C')

For observations:  [1 1 1 2 3]
Solution by Weighted Sampling Algorithm after 50000 samples: ('C', 'C', 'C', 'H', 'H')

For observations:  [1 3 1 1]
Solution by Weighted Sampling Algorithm after 50000 samples: ('C', 'C', 'C', 'C')

For observations:  [3 2 3]
Solution by Weighted Sampling Algorithm after 50000 samples: ('H', 'H', 'H')

For observations:  [2 2 1 1]
Solution by Weighted Sampling Algorithm after 50000 samples: ('H', 'W', 'C', 'C')

For observations:  [1 3 2 3 3]
Solution by Weighted Sampling Algorithm after 50000 samples: ('C',

### Get result of <font color = "red">Forward-backward </font>algorithm

In [114]:
for i in range(len(data)):
    obs = data[i][data[i] != 0]
    print ("For observations: ", obs)
    predict_H = calculate_prob(1, "H", obs, new_hidden_variables, new_start_prob, new_transition_prob, new_emission_prob)
    predict_C = calculate_prob(1, "C", obs, new_hidden_variables, new_start_prob, new_transition_prob, new_emission_prob)
    predict_W = calculate_prob(1, "W", obs, new_hidden_variables, new_start_prob, new_transition_prob, new_emission_prob)
    print ("Probability of p(x1='H'|x) is: ", predict_H)
    print ("Probability of p(x1='C'|x) is: ", predict_C)
    print ("Probability of p(x1='W'|x) is: ", predict_W)
    values = [predict_H, predict_C, predict_W]
    values.sort()
    if values[0] == predict_H:
        print("By comparison, the state is more likely to be: H")
    elif values[0] == predict_C:
        print("By comparison, the state is more likely to be: C")
    else:
        print("By comparison, the state is more likely to be: W")
    print ()
        

For observations:  [2 3 3 2 3]
Probability of p(x1='H'|x) is:  0.4933620487399238
Probability of p(x1='C'|x) is:  0.045513354995408394
Probability of p(x1='W'|x) is:  0.46112459626466795
By comparison, the state is more likely to be: C

For observations:  [2 3 2 2]
Probability of p(x1='H'|x) is:  0.414204372813278
Probability of p(x1='C'|x) is:  0.02327325730517469
Probability of p(x1='W'|x) is:  0.5625223698815472
By comparison, the state is more likely to be: C

For observations:  [3 1 3 3 1]
Probability of p(x1='H'|x) is:  0.0006236190472842495
Probability of p(x1='C'|x) is:  0.7023862684191963
Probability of p(x1='W'|x) is:  0.29699011253351953
By comparison, the state is more likely to be: H

For observations:  [2 1 1]
Probability of p(x1='H'|x) is:  0.0006656604785025886
Probability of p(x1='C'|x) is:  0.56115369985947
Probability of p(x1='W'|x) is:  0.43818063966202747
By comparison, the state is more likely to be: H

For observations:  [1 1 1 2 3]
Probability of p(x1='H'|x) is:

## Conclusion: After adding the third state "Warm", the result becomes smoother since we have a relative resonable state between "cold" and "hot"